In [1]:
import pandas as pd 
import numpy as np
from os.path import dirname, join
import os
#pd.set_option("max_rows", None)

In [2]:
#function to call merged_gw.csv file in every season folder
def import_merged_gw(season='2020-21'):
    path = os.getcwd()
    filename = 'merged_gw.csv'
    season_path = join(dirname(dirname("__file__")), path, 'data', season, 'gws', filename)
    
    return season_path

In [3]:
season_latin = ['2016-17', '2017-18', '2018-19', '2019-20', '2020-21'] 
encoding_latin = ['latin-1', 'latin-1', 'latin-1', 'utf-8', 'utf-8']

df = pd.DataFrame()
for i,j in zip(season_latin, encoding_latin):
    data = pd.read_csv(import_merged_gw(season=f'{i}'), encoding=f'{j}')
    data['season'] = i
    df = df.append(data, ignore_index=True, sort=False)

df = df[['season','name', 'position', 'team', 'assists','bonus','bps','clean_sheets','creativity','element','fixture','goals_conceded','goals_scored','ict_index','influence','kickoff_time','minutes','opponent_team','own_goals','penalties_missed','penalties_saved','red_cards','round','saves','selected','team_a_score','team_h_score','threat','total_points','transfers_balance','transfers_in','transfers_out','value','was_home','yellow_cards','GW']]  

In [4]:
def clean_players_name_string(df, col='name'):
    #replace _ with space in name column
    df[col] = df[col].str.replace('_', ' ')
    #remove number in name column
    df[col] = df[col].str.replace('\d+', '')
    #trim name column
    df[col] = df[col].str.strip()

    return df

In [5]:
df = clean_players_name_string(df, col='name')

In [6]:
def filter_players_exist_latest(df, col='position'):
    #fill in null position (data that only available in 20-21 season) into previous seasons. Null meaning that player doesnt exist in latest season
    df[col] = df.groupby('name')[col].apply(lambda x: x.ffill().bfill())
    df = df[df[col].notnull()]
    return df

In [7]:
df = filter_players_exist_latest(df, col='position')

In [8]:
def get_opponent_team_name(df):
    path = os.getcwd()
    filename = 'master_team_list.csv'
    team_path = join(dirname(dirname("__file__")), path, 'data', filename)
    df_team = pd.read_csv(team_path)

    #create id column for both df_team and df
    df['id'] = df['season'].astype(str) + '_' + df['opponent_team'].astype(str)
    df_team['id'] = df_team['season'].astype(str) + '_' + df_team['team'].astype(str)

    #merge two dfs
    df = pd.merge(df, df_team, on = 'id', how = 'left')

    #rename column
    df = df.rename(columns={"team_name": "opp_team_name"})

    return df

In [9]:
df = get_opponent_team_name(df)

In [10]:
df = df[['season_x', 'name', 'position', 'team_x', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW']]

In [11]:
df.to_csv('cleaned_merged_seasons.csv', encoding = 'utf-8')